# Find the invariant mass of the Z boson!

In [1]:
import ROOT
from ROOT import TMath

Welcome to JupyROOT 6.26/02


In order to activate visualisation of the histogram that is later created we can use the JSROOT magic:

In [2]:
%jsroot on

Next, we have to open the data that we want to analyze. The data is stored in a *.root file

In [3]:
## CHOOSE here which sample to use!!

## 2lep
f = ROOT.TFile.Open("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/1largeRjet1lep/MC/mc_361106.Zee.1largeRjet1lep.root")

After the data is opened, we create a canvas on which we can draw a histogram. If we do not have a canvas we cannot see our histogram at the end. Its name is Canvas and its header is c. The two following arguments define the width and the height of the canvas.

In [4]:
canvas = ROOT.TCanvas("Canvas", "c", 800, 600)

In [5]:
tree = f.Get("mini")
tree.GetEntries()

53653

Now, we define a histogram that will later be place on this canvas. Its name is variable, the header of the histogram is Mass of the Z boson, the x axis is named mass[GeV] and the y axis is name events, The three following arguments indicate that this histogram contains 30 bins which have a range from 40 to 140.

In [6]:
hist = ROOT.TH1F("variable","Mass of the Z boson; mass [GeV]; events",30,40,140)

Time to fill our histogram defined above. At first, we define some variables and then we loop over the data. We also make some cuts as you can see in the # comments.

In [7]:
#leadingLepton = ROOT.TLorentzVector()
#secondLeadingLepton = ROOT.TLorentzVector()

In [8]:
for event in tree:
    
    #Cut #1: At least 2 leptons
    if tree.lep_n >= 2:
        
        #Cut #2: Leptons with opposite charge
        if(tree.lep_charge[0] != tree.lep_charge[1]):
            
            #Cut #3: Leptons of the same family (2 electrons or 2 muons)
            if(tree.lep_type[0] == tree.lep_type[1]):
 
                sumE = tree.lep_E[0] + tree.lep_E[1]
                
                px_leading = tree.lep_pt[0]*TMath.Cos(tree.lep_phi[0])
                px_subleading = tree.lep_pt[1]*TMath.Cos(tree.lep_phi[1])
                
                py_leading = tree.lep_pt[0]*TMath.Sin(tree.lep_phi[0])
                py_subleading = tree.lep_pt[1]*TMath.Sin(tree.lep_phi[1])
                
                pz_leading = tree.lep_pt[0]*TMath.SinH(tree.lep_eta[0])
                pz_subleading = tree.lep_pt[1]*TMath.SinH(tree.lep_eta[1])    
                
                # sumpx = sum of x-momenta
                sumpx = px_leading + px_subleading
                
                # sumpy = sum of y-momenta
                sumpy = py_leading + py_subleading
                
                # sumpz = sum of z-momenta
                sumpz = pz_leading + pz_subleading
                
                # sump = magnitude of total momentum vector.
                sump = TMath.Sqrt(TMath.Power(sumpx,2) + TMath.Power(sumpy,2) + TMath.Power(sumpz,2))
                
                # Mll = invariant mass from M^2 = E^2 - p^2
                Mll = TMath.Sqrt((TMath.Power(sumE,2) - TMath.Power(sump,2)))/1000.
                #print(Mll)
                
                hist.Fill(Mll)
                


After filling the histogram we want to see the results of the analysis. First, we draw the histogram on the canvas and then the canvas on which the histogram lies.

In [9]:
hist.Draw()
hist.SetFillColor(3)

In [10]:
canvas.Draw()
print(hist.GetMaximum())

13224.0


Now try to fit the mass resonance of the W boson using the "Fit" procedure from ROOT. For this we will use the Breit-Wigner Function

In [11]:
# Define Breit-Wigner function
fBW = ROOT.TF1("fBW", "[0]/((x-[1])*(x-[1])+([2]*[2])/4)", 60, 120)

# Set initial parameter values
fBW.SetParameter(0, hist.GetMaximum()) # Overall normalization
fBW.SetParameter(1, 91.2) # Mass of the Z boson
fBW.SetParameter(2, 2.5)  # Width of the Z boson resonance

''' Using directly the Breit Wigner functions from TMath (ROOT) library

# Set up the fit function
def fit_function(x, par):
    return par[0]*ROOT.TMath.BreitWigner(x[0], par[1], par[2])
    
# Create a TF1 object for the fit function
bw = ROOT.TF1("bw", fit_function, 60, 120, 3)

# Set initial parameter values for the fit
bw.SetParameters(100000, 91.2, 2.5)
'''

# Fit histogram with Breit-Wigner function
hist.Fit("fBW","RS")

 FCN=153.745 FROM MIGRAD    STATUS=CONVERGED     155 CALLS         156 TOTAL
                     EDM=2.79337e-10    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           7.96336e+04   8.68241e+02   2.66656e+00  -3.17650e-08
   2  p1           9.04930e+01   1.61422e-02   8.92953e-05   1.04220e-03
   3  p2           4.28277e+00   4.35057e-02   1.27991e-04   2.22388e-04


In [12]:
fBW.Draw("sames")

In [13]:
canvas.Draw()

In [14]:
hist.GetYaxis().SetRangeUser(0, hist.GetMaximum()+0.4*hist.GetMaximum())

In [15]:
canvas.Draw()